In [1]:
import tensorflow as tf
from sklearn.datasets import fetch_lfw_people
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [2]:
# Load LFW dataset
lfw = fetch_lfw_people(min_faces_per_person=50, resize=0.4)
X = lfw.images
y = lfw.target
target_names = lfw.target_names
num_classes = len(target_names)

In [3]:
# Reshape and normalize
X = X[..., tf.newaxis] / 255.0  # (n, h, w, 1)
y_cat = to_categorical(y, num_classes)

In [4]:
# Split data
X_train, X_val, y_train, y_val = train_test_split(X, y_cat, test_size=0.2, random_state=42)

In [5]:
# Build model
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=X.shape[1:]),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

In [6]:
# Compile and train
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val))

# Save model
model.save("cnn_face_recognition.h5")

# Save label names
with open("class_labels.txt", "w") as f:
    for name in target_names:
        f.write(f"{name}\n")
        

Epoch 1/10
39/39 [==============================] - 5s 54ms/step - loss: 2.2536 - accuracy: 0.3341 - val_loss: 2.0861 - val_accuracy: 0.3429
Epoch 2/10
39/39 [==============================] - 2s 54ms/step - loss: 2.1664 - accuracy: 0.3389 - val_loss: 2.0944 - val_accuracy: 0.3429
Epoch 3/10
39/39 [==============================] - 2s 55ms/step - loss: 2.1552 - accuracy: 0.3389 - val_loss: 2.0848 - val_accuracy: 0.3429
Epoch 4/10
39/39 [==============================] - 2s 55ms/step - loss: 2.1523 - accuracy: 0.3389 - val_loss: 2.0991 - val_accuracy: 0.3429
Epoch 5/10
39/39 [==============================] - 2s 52ms/step - loss: 2.1551 - accuracy: 0.3389 - val_loss: 2.0849 - val_accuracy: 0.3429
Epoch 6/10
39/39 [==============================] - 2s 54ms/step - loss: 2.1530 - accuracy: 0.3389 - val_loss: 2.0871 - val_accuracy: 0.3429
Epoch 7/10
39/39 [==============================] - 2s 55ms/step - loss: 2.1560 - accuracy: 0.3389 - val_loss: 2.0797 - val_accuracy: 0.3429
Epoch 8/10
39

In [1]:
import cv2
import numpy as np
import tensorflow as tf

# Load trained model
model = tf.keras.models.load_model(r"D:\Coding\Projects\facial-recognition\scripts\cnn_face_recognition.h5")

# Load class labels
with open(r"D:\Coding\Projects\facial-recognition\scripts\class_labels.txt", "r") as f:
    class_names = [line.strip() for line in f]

# Initialize webcam
cap = cv2.VideoCapture(0)

# Constants (LFW images are 50x37)
IMG_HEIGHT, IMG_WIDTH = 50, 37

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Convert to grayscale (LFW dataset is grayscale)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Use Haar cascade to detect faces
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)

    for (x, y, w, h) in faces:
        face_img = gray[y:y+h, x:x+w]
        face_resized = cv2.resize(face_img, (IMG_WIDTH, IMG_HEIGHT))
        face_input = face_resized.reshape(1, IMG_HEIGHT, IMG_WIDTH, 1) / 255.0

        # Predict using CNN
        preds = model.predict(face_input)
        class_idx = np.argmax(preds)
        confidence = preds[0][class_idx]

        # Draw bounding box and label
        label = f"{class_names[class_idx]} ({confidence*100:.1f}%)"
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

    # Show output
    cv2.imshow("Face Recognition", frame)

    # Exit on pressing 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Cleanup
cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 31ms/step
